In [2]:
import pandas as pd
import xarray as xr
import numpy as np
import copy
import math
import random
import warnings
import matplotlib.pyplot as plt

import calliope
from calliope.exceptions import ModelWarning

calliope.set_log_verbosity(verbosity='critical', include_solver_output=False, capture_warnings=False)

# Suppress the specific ModelWarning from Calliope
warnings.filterwarnings("ignore", category=ModelWarning)

from deap import base
from deap import creator
from deap import tools

In [4]:
model = calliope.Model("C:/Users/Jacob/Desktop/PythonProjects/GAMGA-Calliope v3.9/GAMGA_model/model_14D.yaml")

model.run()

df_total_cost = model.results.cost.to_series().dropna()
total_cost_optimal = df_total_cost.loc[~df_total_cost.index.map(str).str.contains('co2_emissions')].sum()

print(total_cost_optimal)

energy_cap_df = model.results.energy_cap.to_pandas()
filtered_energy_cap_df = energy_cap_df[~energy_cap_df.index.str.contains("demand|transmission")]

print(filtered_energy_cap_df)


29759791.727324426
loc_techs
region1::ccgt         5000.000000
region1::solarPV1        0.000027
region1::battery1        0.000276
region2::battery2     2355.664275
region2::solarPV2    10321.864197
dtype: float64


In [8]:
#create the model with a resolution of 6 hours
model_6H = calliope.Model('C:/Users/Jacob/Desktop/PythonProjects/GAMGA-Calliope v3.9/GAMGA_model/model_6H.yaml')

model_6H.run()

df_total_cost_6H = model_6H.results.cost.to_series().dropna()
total_cost_optimal_6H = df_total_cost_6H.loc[~df_total_cost_6H.index.map(str).str.contains('co2_emissions')].sum()

print(total_cost_optimal_6H)

energy_cap_df_6H = model_6H.results.energy_cap.to_pandas()
filtered_energy_cap_df_6H = energy_cap_df_6H[~energy_cap_df_6H.index.str.contains("demand|transmission")]

print(filtered_energy_cap_df_6H)

31968422.012815084
loc_techs
region1::ccgt         4999.999999
region1::solarPV1     1797.625891
region1::battery1        0.000777
region2::battery2     4636.735890
region2::solarPV2    11079.677565
dtype: float64


In [7]:
model.backend.update_param('energy_cap_max', {'region1::solarPV1': 0.000027})
model.backend.update_param('energy_cap_min', {'region1::solarPV1': 0.000027})
model.backend.update_param('energy_cap_max', {'region1::battery1': 0.000276})
model.backend.update_param('energy_cap_min', {'region1::battery1': 0.000276})
model.backend.update_param('energy_cap_max', {'region2::solarPV2': 10321.864197})
model.backend.update_param('energy_cap_min', {'region2::solarPV2': 10321.864197})
model.backend.update_param('energy_cap_max', {'region2::battery2': 2355.664275})
model.backend.update_param('energy_cap_min', {'region2::battery2': 2355.664275})
model.backend.update_param('energy_cap_max', {'region1::ccgt': 1000.000000})
model.backend.update_param('energy_cap_min', {'region1::ccgt': 1000.000000})

In [9]:
model_6H.backend.update_param('energy_cap_max', {'region1::solarPV1': 1797.625891})
model_6H.backend.update_param('energy_cap_min', {'region1::solarPV1': 1797.625891})
model_6H.backend.update_param('energy_cap_max', {'region1::battery1': 0.000777})
model_6H.backend.update_param('energy_cap_min', {'region1::battery1': 0.000777})
model_6H.backend.update_param('energy_cap_max', {'region2::solarPV2': 11079.677565})
model_6H.backend.update_param('energy_cap_min', {'region2::solarPV2': 11079.677565})
model_6H.backend.update_param('energy_cap_max', {'region2::battery2': 4636.735890})
model_6H.backend.update_param('energy_cap_min', {'region2::battery2': 4636.735890})
model_6H.backend.update_param('energy_cap_max', {'region1::ccgt': 1000.999999})
model_6H.backend.update_param('energy_cap_min', {'region1::ccgt': 1000.999999})

In [10]:
rerun_model = model.backend.rerun()  # Rerun to capture updated backend parameters
rerun_model_6H = model_6H.backend.rerun()

In [11]:
# Calculate the total cost, excluding emission costs
cost_op = rerun_model.results.cost.to_series().dropna()
initial_cost = round(cost_op.loc[~cost_op.index.map(str).str.contains('co2_emissions')].sum(), 2)

# unmet = rerun_model.results.unmet_demand.to_series().dropna()
# unmet_demand = round(unmet.sum() * 1, 2) #300 is the penalty for unmet demand

total_cost = initial_cost #+ unmet_demand

print(total_cost)

11303345.93


In [12]:
# Calculate the total cost, excluding emission costs
cost_op = rerun_model_6H.results.cost.to_series().dropna()
initial_cost = round(cost_op.loc[~cost_op.index.map(str).str.contains('co2_emissions')].sum(), 2)

# unmet = rerun_model_6H.results.unmet_demand.to_series().dropna()
# unmet_demand = round(unmet.sum() * 1, 2) #300 is the penalty for unmet demand


total_cost = initial_cost #+ unmet_demand

print(total_cost)

13525565.08
